In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import seaborn as sns
import plotly.express as px

In [ ]:
df = pd.read_csv("flightsCleaned.csv")

# Añadimos el FlightNum como variable categórica
df['FLIGHT_NUMBER']=df['FLIGHT_NUMBER'].astype(object) 
# Configuramos DATE como variable de tipo datetime
df['DATE'] = pd.to_datetime(df['DATE'])

pd.set_option('display.max_columns', None)
df.head()

In [ ]:
df.shape

Quitamos filas con códigos de aeropuerto desconocidos

In [ ]:
df.isna().sum()


In [ ]:
df.dropna()

In [ ]:
df["ORIGIN_STATE"].unique()

## Análisis Descriptivo

Primero hacemos un análisis de los retrasos a lo largo del año

In [ ]:
del data
data = pd.DataFrame()
data["Vuelos Totales"] = df.groupby('DATE')["FLIGHT_NUMBER"].count()
data["Vuelos Retrasados"] = df[df["ARRIVAL_DELAY"]>0].groupby('DATE')["FLIGHT_NUMBER"].count()
data.head()

In [ ]:
fig = px.line(data, x=data.index, y=["Vuelos Totales", "Vuelos Retrasados"], color_discrete_sequence=px.colors.qualitative.Vivid)
fig.update_layout(
    title="Tendencia vuelos totales y retrasados a lo largo del año",
    xaxis_title="Fecha",
    yaxis_title="Vuelos",
    legend_title="Leyenda",
    template="plotly_dark",
    hovermode="x unified"
)
fig.show()

Parece haber una tendencia semanal, vamos a confirmarlo

In [ ]:
## Definimos la paleta usada con plotly express, para poder utilizarla con plotly normal
color_palette = px.colors.qualitative.Vivid

In [ ]:
del data
data = pd.DataFrame()
data["Vuelos Totales"] = df.groupby(df['DATE'].dt.day_name())["FLIGHT_NUMBER"].count()
data["Vuelos Retrasados"] = df[df["ARRIVAL_DELAY"]>0].groupby(df['DATE'].dt.day_name())["FLIGHT_NUMBER"].count()
data = data.reindex(index = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
data.head()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=data.index, y=data["Vuelos Totales"], name="Totales", marker_color = color_palette[0]))
fig.add_trace(go.Scatter(x=data.index, y=data["Vuelos Retrasados"], name="Retrasados", line_color=color_palette[1]))
fig.update_layout(
    title="Vuelos totales y retrasados en función de la aerolinea",
    xaxis_title="Día de la semana",
    yaxis_title="Vuelos",
    legend_title="Leyenda",
    template="plotly_dark",
    hovermode="x unified"
)
fig.show()

Efectivamente, los sábados hay menos vuelos

Lo siguiente será analizar cuánto se retrasan de media los vuelos y debido a qué causas, a lo largo del año

In [ ]:
data = df.groupby(df['DATE'].dt.month_name()).mean()
ORDERED_MONTHS = ["January", "February", "March", "April", "May", "June",
      "July", "August", "September", "October", "November", "December"]
data = data.reindex(index = ORDERED_MONTHS)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data["AIR_SYSTEM_DELAY"], stackgroup='one', name="Sistema aéreo", line_color=color_palette[0]))
fig.add_trace(go.Scatter(x=data.index, y=data["SECURITY_DELAY"], stackgroup='one', name="Seguridad", line_color=color_palette[1])) 
fig.add_trace(go.Scatter(x=data.index, y=data["AIRLINE_DELAY"], stackgroup='one', name="Aerolínea", line_color=color_palette[2])) 
fig.add_trace(go.Scatter(x=data.index, y=data["LATE_AIRCRAFT_DELAY"], stackgroup='one', name="Avión retrasado", line_color=color_palette[3])) 
fig.add_trace(go.Scatter(x=data.index, y=data["WEATHER_DELAY"], stackgroup='one', name="Tiempo meteorológico", line_color=color_palette[4])) 
fig.add_trace(go.Scatter(x=data.index, y=data["OTHER_DELAY"], stackgroup='one', name="Otro", line_color=color_palette[5])) 

fig.update_layout(
    title="Retraso medio y causa a lo largo del año",
    xaxis_title="Mes",
    yaxis_title="Minutos",
    legend_title="Causa del Retraso",
    template="plotly_dark",
    hovermode="x unified"
)

fig.show()

¿Correlación entre distancia y retraso?

In [ ]:
fig = px.scatter(df[df["ARRIVAL_DELAY"]>0], x="ARRIVAL_DELAY", y="DISTANCE", color_discrete_sequence=px.colors.qualitative.Vivid)
fig.update_layout(
    title="Tendencia vuelos totales y retrasados a lo largo del año",
    xaxis_title="Fecha",
    yaxis_title="Vuelos",
    legend_title="Leyenda",
    template="plotly_dark"
)
fig.show()

No hay correlación entre la distancia y el retraso de los vuelos

In [ ]:
df.columns

In [ ]:
df[["ORIGIN_LATITUDE","ORIGIN_LONGITUDE"]]

In [ ]:
df[["ORIGIN_AIRPORT","ORIGIN_CITY"]]

In [ ]:
airports = df[["ORIGIN_AIRPORT","ORIGIN_CITY","ORIGIN_LATITUDE","ORIGIN_LONGITUDE"]].groupby(["ORIGIN_AIRPORT","ORIGIN_CITY"]).mean()
airports["FLIGHTS"] = df.groupby(["ORIGIN_AIRPORT","ORIGIN_CITY"]).size()
airports["DELAYED_FLIGHTS"] = df[df["ARRIVAL_DELAY"]>0].groupby(["ORIGIN_AIRPORT","ORIGIN_CITY"]).size()
airports["DELAYED_PERCENTAGE"] = airports["DELAYED_FLIGHTS"]/airports["FLIGHTS"]
airports = airports.reset_index()
airports.head()

In [ ]:
fig = px.scatter_geo(airports, lat="ORIGIN_LATITUDE", lon = "ORIGIN_LONGITUDE",
                     size= "FLIGHTS", # size of markers
                     color= "DELAYED_PERCENTAGE", # which column to use to set the color of markers
                     scope="usa",
                     text = "ORIGIN_AIRPORT",
                     hover_data  = ["ORIGIN_CITY"],
                     color_continuous_scale='RdYlGn_r',
                     template="plotly_dark")
fig.update_traces(textposition="top center")
fig.update_layout(
    title="Aeropuertos de origen con número de vuelos y porcentaje de retrasos",
    legend_title="Causa del Retraso",
    width = 1000, 
    height = 650
)
fig.show()

In [ ]:
del data
data = pd.DataFrame()
data["Vuelos Totales"] = df.groupby(df['DATE'])["FLIGHT_NUMBER"].count()
data["Vuelos Retrasados"] = df[df["ARRIVAL_DELAY"]>0].groupby(df['DATE'])["FLIGHT_NUMBER"].count()
data.head()

## EDA

In [ ]:
# Variables categoricas
df.select_dtypes('object').nunique()

In [ ]:
# Variables numéricas
df.describe()

In [ ]:
# Correlación entre variables númericamente
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(4)

**NOTA:** cambiar colores :)

In [ ]:
# Estudio de las fechas --> ¿Algún mes en el que se produzcan más retrasos?
# Querría hacer una matriz: filas  = días del mes | columnas = meses | valor = numero de vuelos retrasados
# Plot retrasos por día del mes

In [ ]:
# Comparación razones de retraso

### Estudio origen y destino

1) mapa geográfico térmico (colo en función de salidas retrasadas)

2) rutas qué experimentan más retrasos

In [ ]:
print("Numero de vuelos repetidos: " +str(len(df['FlightNum'].unique()))+ " de un total de "+ str(df.size))

In [ ]:
# Comprobar rutas de los vuelos repetidos

### Carriers

In [ ]:
# Visualizar carriers

# Causas cese
level_count = df["AIRLINE"].value_counts()
level_count

level_df = pd.DataFrame(level_count).reset_index().rename(columns = {"index": "Carrier", "DelayedFlights": "count"})
level_count.sort_values(ascending = True,inplace = True) # Reordeno para que me quede de mayor a menor el gráfico

data = [
    go.Bar(
        y = level_count.index,
        x = level_count,
        name = "Vuelos retrasados por Compañia",
        marker_color = "cadetblue",
        width= np.repeat(0.65,len(level_count)), # Cambiar la anchura de cada barra,
        orientation = 'h'
    )
]

layout = go.Layout(title = "Vuelos retrasados por Compañia", yaxis_title = "Carriers", 
                   xaxis_title = "Número de vuelos retrasados",
                  )

fig = go.Figure(data = data, layout = layout)

fig.show()


Dentro de las compañias que sufren más retrasos, mostrar:
- rango de retraso salida
- rango de retraso llegada


### Caso anterior con aeropuertos de salida

## Cinco aeropuertos con mayor retraso

Definir si el aeropuerto con mayor retraso es en números aboslutos o comparado con la cantidad de vuelos que operan por aeropuerto --> % respecto a la cantidad de vuelos que opera

In [ ]:
# Representación de los aeropuetos
airports = pd.read_csv("airports.csv")
pd.set_option('display.max_columns', None)
airports.head()

In [ ]:
airports.columns

In [ ]:
airports['COUNTRY'].unique()

http://www.geomapik.com/desarrollo-programacion-gis/mapas-plotly-visualizacion-datos-espaciales/

In [ ]:
# Agrupar por fechas
#flights[flights["ORIGIN_AIRPORT"] == "LAX"].groupby(["YEAR","MONTH"]).size()